In [241]:
#import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import random
import os

**Read December 24 data into Dataframe**

In [244]:
# # Read december 24 provider data from excel and create a dataframe called as dec_24_provider
# dec_24_provider = pd.read_excel('../FL_mental_health_providers/data/EHR_GEHRIMED_C4631G6561_2024-12-31_Facility and Provider GDR Report_Monthly.xlsx' , sheet_name='Provider MTD')
# dec_24_provider

In [246]:
# # define a function to randomize Provider Name values
# def randomize_values(col_values):
#     col_values_list = list(col_values) # convert string to list
#     random.shuffle(col_values_list)
#     return ''.join(col_values_list) # convert list back to string

In [248]:
# # apply the function to the Provider Name
# column_to_randomize = 'Provider Name'
# dec_24_provider[column_to_randomize] = dec_24_provider[column_to_randomize].apply(randomize_values)
# dec_24_provider

**Yearly report of Florida mental health provider data countywise from year 16-17 to year 22-23**

In [251]:
# Read Ten years Florida provider report from excel and create a dataframe called as ten_year_report
ten_year_report = pd.read_excel('../FL_mental_health_providers/data/TenYrsReport.xlsx', skiprows=2)
ten_year_report.head()

,County,Count,Rate,Count.1,Rate.1,Count.2,Rate.2,Count.3,Rate.3,Count.4,...,Count.6,Rate.6,Count.7,Rate.7,Count.8,Rate.8,Count.9,Rate.9,Unnamed: 21,Unnamed: 22
0,Florida,30213,133.2,29121,130.4,27174,123.5,25340,117.1,23453,...,20542,99.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alachua,713,244.6,722,248.4,676,245.4,644,238.2,638,...,590,227.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Baker,21,75.6,20,68.5,18,62.5,18,63.0,15,...,15,55.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,County
3,Bay,245,130.4,232,128.3,194,109.7,186,105.8,188,...,179,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FY_22_23_Count
4,Bradford,19,71.2,17,61.1,15,52.0,14,48.6,11,...,5,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FY_22_23_Rate


In [253]:
#ten_year_report.info()

In [255]:
# Drop the count and blank columns
ten_year_report = ten_year_report.drop(columns=['Count','Count.1','Count.2','Count.3','Count.4','Count.5','Count.6','Count.7','Rate.7','Count.8','Rate.8','Count.9','Rate.9','Unnamed: 21','Unnamed: 22'], axis=1)
#ten_year_report

In [257]:
# Rename appropriate column names of ten_year_report dataframe 
ten_year_report.columns = ['County','FY_22_23','FY_21_22','FY_20_21','FY_19_20','FY_18_19','FY_17_18','FY_16_17']
#ten_year_report

In [259]:
# Reorder columns from the year 16-17 to year 22-23
ten_year_report = ten_year_report.reindex(columns=['County','FY_16_17','FY_17_18','FY_18_19','FY_19_20','FY_20_21','FY_21_22','FY_22_23'])
#ten_year_report = ten_year_report.reindex(columns={'County':'County','Rate.6':'FY_16_17', 'Rate.5': 'FY_17_18', 'Rate.4': 'FY_18_19','Rate.3':'FY_19_20','Rate.2':'FY_20_21','Rate.1':'FY_21_22','Rate':'FY_22_23'})
ten_year_report

,County,FY_16_17,FY_17_18,FY_18_19,FY_19_20,FY_20_21,FY_21_22,FY_22_23
0,Florida,99.9,104.6,110.3,117.1,123.5,130.4,133.2
1,Alachua,227.5,226.7,239.3,238.2,245.4,248.4,244.6
2,Baker,55.4,58.2,53.4,63.0,62.5,68.5,75.6
3,Bay,100.0,92.7,104.5,105.8,109.7,128.3,130.4
4,Bradford,18.0,32.0,38.7,48.6,52.0,61.1,71.2
...,...,...,...,...,...,...,...,...
64,Volusia,89.5,90.1,95.6,97.5,100.9,116.1,121.2
65,Wakulla,68.5,71.1,80.2,83.8,89.0,105.6,102.7
66,Walton,74.6,82.4,88.1,97.9,92.0,111.2,102.5
67,Washington,36.1,43.6,47.3,55.4,62.7,64.1,61.8


In [261]:
# Convert the year labels into a dataframe column named "Year"
ten_year_report = ten_year_report.melt(id_vars=["County"]
        , var_name="Year")
        #, value_name="Rate")

In [263]:
# fetch the current working directory
cwd = os.getcwd() 
# select the data folder to save the csv file
path = cwd + "\\data" 
# define the file name for csv
filename = 'ten_year_report.csv'
# Create the full file path
csv_path = os.path.join(path, filename)

In [265]:
# convert the DataFrame to CSV file and save it in given path
ten_year_report.to_csv(csv_path, index=False) 